In [1]:
import tkinter as tk
import threading
import time
from queue import Queue
import requests
import folium
import openrouteservice as ors
from IPython.display import display  # For map display in Jupyter-like environments


class NodeHubSystem:
    def __init__(self):
        self.node_queues = [Queue() for _ in range(3)]  # Queues for Node 1, Node 2, Node 3
        self.hub_queue = Queue()  # Queue for the Hub
        self.sent_alerts = set()
        self.node_windows = []
        self.alert_generated = False  # Track if an alert has been generated

        # Source and destination locations (geocoded coordinates for Beeramguda and Kukatpally)
        self.source = [78.3033, 17.5019]  # Beeramguda (approximate coordinates)
        self.destination = [78.4138, 17.4949]  # Kukatpally (approximate coordinates)

        # Initialize OpenRouteService client
        self.client = ors.Client(key='5b3ce3597851110001cf6248f71e4da02d6f409bb1bc92a59f0d107e')

    def start(self):
        # Create windows for nodes and hub
        for i in range(3):
            threading.Thread(target=self.create_node_window, args=(i,), daemon=True).start()

        threading.Thread(target=self.create_hub_window, daemon=True).start()

        # Start monitoring ThingSpeak channels
        threading.Thread(target=self.monitor_sensors, daemon=True).start()

        # Start the Tkinter GUI to show the map
        threading.Thread(target=self.create_map_window, daemon=True).start()

    def create_node_window(self, node_id):
        def process_queue():
            while not self.node_queues[node_id].empty():
                message = self.node_queues[node_id].get()
                text_widget.insert(tk.END, message + "\n")
                text_widget.see(tk.END)
            root.after(100, process_queue)

        root = tk.Tk()
        root.title(f"Node {node_id + 1} Console")
        text_widget = tk.Text(root, height=20, width=50)
        text_widget.pack()

        process_queue()
        root.mainloop()

    def create_hub_window(self):
        def process_queue():
            while not self.hub_queue.empty():
                message = self.hub_queue.get()
                text_widget.insert(tk.END, message + "\n")
                text_widget.see(tk.END)
            root.after(100, process_queue)

        root = tk.Tk()
        root.title("Hub Console")
        text_widget = tk.Text(root, height=20, width=50)
        text_widget.pack()

        process_queue()
        root.mainloop()

    def create_map_window(self):
        def update_map():
            self.generate_map()
            root.after(5000, update_map)  # Refresh map every 5 seconds

        root = tk.Tk()
        root.title("Map Display")
        root.geometry("800x600")

        map_frame = tk.Frame(root)
        map_frame.pack(fill=tk.BOTH, expand=True)

        update_map()
        root.mainloop()

    def monitor_sensors(self):
        # ThingSpeak Channel Details
        channels = [
            {"id": "2784770", "api_key": "I6I38IODBRXVZCAV"},
            {"id": "2795362", "api_key": "225NJ2MKZB1U4RH0"},
            {"id": "2795423", "api_key": "ZZJLS0LI3ZUS7W84"},
        ]

        while True:
            sensor_data = [
                self.fetch_last_values(channel["id"], channel["api_key"], 8)
                for channel in channels
            ]

            alert_conditions = [self.check_alert_conditions(data) for data in sensor_data]

            self.handle_alerts(alert_conditions)
            time.sleep(5)

    def fetch_last_values(self, channel_id, api_key, field_count):
        url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.json"
        params = {"api_key": api_key, "results": 1}
        response = requests.get(url, params=params)

        if response.status_code == 200:
            data = response.json()
            if data and "feeds" in data and len(data["feeds"]) > 0:
                last_entry = data["feeds"][-1]
                sensor_data = {}
                for field_num in range(1, field_count + 1):
                    sensor_data[f"field{field_num}"] = last_entry.get(f"field{field_num}")
                return sensor_data
            else:
                return None
        else:
            return None

    def check_alert_conditions(self, sensor_data):
        if sensor_data:
            field_6 = float(sensor_data.get("field6", 0))
            field_7 = float(sensor_data.get("field7", 0))
            field_8 = float(sensor_data.get("field8", 0))

            return field_6 < 5 and field_7 == 0 and field_8 == 0
        return False

    def handle_alerts(self, alert_conditions):
        nodes_with_alerts = [i for i, condition in enumerate(alert_conditions) if condition]

        if len(nodes_with_alerts) > 0:
            alert_key = tuple(sorted(nodes_with_alerts))

            if alert_key not in self.sent_alerts:
                message = f"Accident occurred at Nodes: {', '.join(f'Node {n + 1}' for n in nodes_with_alerts)}!"
                self.hub_queue.put(message)
                self.sent_alerts.add(alert_key)

                for i, queue in enumerate(self.node_queues):
                    if i not in nodes_with_alerts:
                        queue.put(message)

                # Set the alert flag and display the second alternative route
                self.alert_generated = True
        else:
            self.alert_generated = False  # No alert, show default route

    def generate_map(self):
        # Create a map centered at the source location
        m = folium.Map(location=list(reversed(self.source)), tiles="cartodbpositron", zoom_start=14)

        # Add markers for source and destination
        folium.Marker(location=list(reversed(self.source)), icon=folium.Icon(color="red", icon="cloud"), popup="Beeramguda").add_to(m)
        folium.Marker(location=list(reversed(self.destination)), icon=folium.Icon(color="blue", icon="flag"), popup="Kukatpally").add_to(m)

        # Request two alternative routes from OpenRouteService
        routes = self.client.directions(
            coordinates=[self.source, self.destination],
            profile='driving-car',
            alternative_routes={"share_factor": 0.5, "target_count": 2},  # Request two alternative routes
            geometry=True
        )

        # Extract and represent the main route in red (if no alert)
        main_route_coords = ors.convert.decode_polyline(routes['routes'][0]['geometry'])
        folium.PolyLine(locations=[list(reversed(coord)) for coord in main_route_coords['coordinates']], color='red', weight=5, popup="Main Route").add_to(m)

        # If alert is generated, show the second optimized route in blue
        if self.alert_generated and len(routes['routes']) > 1:
            alternative_route_coords = ors.convert.decode_polyline(routes['routes'][1]['geometry'])
            folium.PolyLine(locations=[list(reversed(coord)) for coord in alternative_route_coords['coordinates']], color='blue', weight=5, popup="Alternative Route").add_to(m)

        display(m)  # This will display the map in Jupyter or a similar environment


# Run the Node-Hub system
if __name__ == "__main__":
    system = NodeHubSystem()
    system.start()
